In [ ]:
import pymysql as sql

f = open('gene.txt',"r")
content = f.readlines()
f.close()

gene_symbol = content[0].split(" ")[0]
gene_id = content[1].split(",")[0].split(":")[1].strip()
print (gene_symbol)
print (gene_id)

indx_lst = [i for i in range(len(content)) if "mRNA transcript" in content[i]]

connection = sql.connect(user='root',password="987654321",host='localhost',port=3306)
db = connection.cursor()
def get_ex_dt(data_content):
    rt = {}
    ex_no = 1
    for dt in data_content:
        ex_dt = dt.split("\t")[0]
        ex_zt = int(ex_dt.split("-")[0])
        ex_ed = int(ex_dt.split("-")[1])
        ex_sz = ex_ed - ex_zt + 1 if ex_ed > ex_zt else ex_zt - ex_ed + 1
        rt[ex_no] = {"st":ex_zt, "ed":ex_ed, "sz":ex_sz}
        ex_no += 1
    return rt

def create_db(db_name):
    db.execute("create database if not exists %s"%(db_name))
    connection.commit()
    
def create_table(db_name,table_name):
    db.execute("use %s"%(db_name))
    db.execute("create table if not exists %s(gene_symbol varchar(100), gene_id varchar(100), trans_id varchar(100), exon_no varchar(100), exon_st varchar(100), exon_ed varchar(100), exon_sz varchar(100))"%(table_name))

def upload_data_to_table(gene_symbol,gene_id,tran_id,exon_no,exon_st,exon_ed,exon_sz,db_name,table_name):
    db.execute("use %s"%(db_name))
    db.execute(f"insert into {table_name} (gene_symbol,gene_id,trans_id,exon_no,exon_st,exon_ed,exon_sz) values ('{gene_symbol}','{gene_id}', '{tran_id}', '{exon_no}', '{exon_st}', '{exon_ed}', '{exon_sz}')")
    connection.commit()
    
if __name__ == "__main__":
    create_db("PYDT_VLCY_PRJ")
    create_table("PYDT_VLCY_PRJ", "GENE_TABLE")
    db_name = "PYDT_VLCY_PRJ"
    table_name = "GENE_TABLE"
    with open("gene_table.csv", "w") as fl:
        fl.write("Gene symbol,Gene ID,Trans ID,Exon No,Exon st,Exon ed,Exon sz")
        for i in range(len(indx_lst)):
            if i != len(indx_lst) - 1:
                st,ed = indx_lst[i],indx_lst[i+1]
                data_content = content[st:ed]
                tr_id = [x for x in data_content[0].split(" ") if "NM_" in x][0]
                tr_id = tr_id.replace(",","")
                
                idx = [i for i in range(len(data_content)) if "--------------" in data_content[i]][0]
                data_content = data_content[idx+1:]
                data_content = [x.strip() for x in data_content if x.strip() != '']
                
                f_d = get_ex_dt(data_content)
                for ex in f_d:
                    fl.write("\n%s,%s,%s,%s,%s,%s,%s"%(gene_symbol, gene_id, tr_id, ex, f_d[ex]["st"],f_d[ex]["ed"],f_d[ex]["sz"]))
                    upload_data_to_table(gene_symbol,gene_id,tr_id, ex, f_d[ex]["st"],f_d[ex]["ed"],f_d[ex]["sz"],db_name,table_name)
                print (tr_id)
                
            else:
                st = indx_lst[i]
                data_content = content[st:]
                tr_id = [x for x in data_content[0].split(" ") if "NM_" in x][0]
                tr_id = tr_id.replace(",","")
                
                idx = [i for i in range(len(data_content)) if "--------------" in data_content[i]][0]
                data_content = [x.strip() for x in data_content if x.strip() != '']
                data_content = data_content[idx+1:]
                
                f_d = get_ex_dt(data_content)
                for ex in f_d:
                    fl.write("\n%s,%s,%s,%s,%s,%s,%s"%(gene_symbol, gene_id, tr_id, ex, f_d[ex]["st"],f_d[ex]["ed"],f_d[ex]["sz"]))
                    upload_data_to_table(gene_symbol,gene_id,tr_id, ex, f_d[ex]["st"],f_d[ex]["ed"],f_d[ex]["sz"],db_name,table_name)
                print (tr_id)
                
db.close()
connection.close()               